# Deep learning framework example: Movie Review Dataset

This notebook demonstrates how to use the deeplearning API to train and test the model on the [Stanford movie review corpus](https://nlp.stanford.edu/sentiment/) corpus.  This dataset contains hand written digits and their labels.  See the [saved version](https://htmlpreview.github.io/?https://github.com/plandes/deepnlp/blob/master/example/movie/notebook/movie.html) for output.

**Important**: Please see the [Clickbate notebook example](https://github.com/plandes/deepnlp/blob/master/example/clickbate/notebook/clickbate.ipynb) for a more simple example of how to utilize the framework to tune pamateres in Jupyter notebooks.

In [ ]:
# environemnt configuration and set up: add this (deepnlp) library to the Python path and framework entry point
from mngfac import JupyterManagerFactory
fac = JupyterManagerFactory()
mng = fac()

## Print information about 

Use the factory to create the model executor.  The `write` method gives statistics on the data set that is configured on the executor.

In [ ]:
facade = mng.create_facade()
mng.write()

# Run the first test

Set the number of epochs to a value that might be different from the default in the configuration files.

In [ ]:
facade.epochs = 15
mng.run()
# uncomment to persist the plot and results to the file system
#facade.persist_result()

## Try word2vec embeddings

Change the embeddings to word2vec and rerun.

In [ ]:
# changing the embedding updates the model, but currently results show initial loaded embeddings (glove50)
facade.embedding = 'word2vec_300_embedding'
mng.run()

# Transformer fixed embeddings

Next try frozen Bert embeddings.  These are computed from the dataset and used as fixed embeddings (rather than being trainable fine-tuned embeddings).  Note the performance isn't much improved since the real strength of a transformer architecture is being able to fine tune it to our sentient task.

In [ ]:
# when comparing several models with the same embeddings with different language
# features (later cells), recreate to get a consistent random seed and clean state
facade = mng.create_facade('transformer-fixed')
# since the configuration files already have a default linguistic feature set, clear it out
facade.language_attributes = set()
mng.run()

In [ ]:
facade = mng.create_facade('transformer-fixed')
facade.language_attributes = {'transformer_enum_expander'}
mng.run()

# Add a transformer expander

A vectorizer that expands lingustic feature vectors to their respective locations as word piece token vectors.  This is used to concatenate lingustic features with Bert (and other transformer) embeddings.  Each lingustic token is copied in the word piece token location across all vectorizers and sentences.

First we'll try the dependency expander, which adds the spaCy head three depth feature to the embeddings.

In [ ]:
facade = mng.create_facade('transformer-fixed')
facade.language_attributes = {'transformer_dep_expander'}
mng.run()

# Add enumerated features

Next try the enumeration expander, which adds the spaCy enumerated parsed values (i.e. POS and NER tags) as features to the corresponding wordpiece tokens.

In [ ]:
facade = mng.create_facade('transformer-fixed')
facade.language_attributes = {'transformer_enum_expander', 'transformer_dep_expander'}
mng.run()

# Bert fine tuning

Finally, try fine tuning the Bert model.

In [ ]:
facade = mng.create_facade('transformer-trainable')
mng.run()

# Adjust optimizer and schedule

Decay the weights in the optimizer and drop the learning rate after 3 validation loss drops.

In [ ]:
facade = mng.create_facade('transformer-trainable')
facade.model_settings.optimizer_params = {'weight_decay': 5e-3}
facade.model_settings.scheduler_params = {'patience': 3}
facade.epochs = 14
mng.run()